In [3]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
import io, s3fs, json, tabula
pd.set_option('display.max_columns', None)
print('Program run at', dt.now())

Program run at 2021-05-01 09:57:35.892580


# Rename ACS columns  
Import ACS data and header data from AWS.

In [2]:
acs = pd.read_csv('s3://bleeding-hearts/rawdata/acs.csv')
acs.head()

/Users/sarahmartin/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (587,588,589,591,592,593,594,595,596,597,598,599,600,601,602,603,604,606,607,608,609,610,611,612,613,615,616,617,618,619,620,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,651,652,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,831,832,835,836,837,838,1057,1058) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Unnamed: 0,GEO_ID,DP03_0001E,DP03_0001M,DP03_0001PE,DP03_0001PM,DP03_0002E,DP03_0002M,DP03_0002PE,DP03_0002PM,DP03_0003E,DP03_0003M,DP03_0003PE,DP03_0003PM,DP03_0004E,DP03_0004M,DP03_0004PE,DP03_0004PM,DP03_0005E,DP03_0005M,DP03_0005PE,DP03_0005PM,DP03_0006E,DP03_0006M,DP03_0006PE,DP03_0006PM,DP03_0007E,DP03_0007M,DP03_0007PE,DP03_0007PM,DP03_0008E,DP03_0008M,DP03_0008PE,DP03_0008PM,DP03_0009E,DP03_0009M,DP03_0009PE,DP03_0009PM,DP03_0010E,DP03_0010M,DP03_0010PE,DP03_0010PM,DP03_0011E,DP03_0011M,DP03_0011PE,DP03_0011PM,DP03_0012E,DP03_0012M,DP03_0012PE,DP03_0012PM,DP03_0013E,DP03_0013M,DP03_0013PE,DP03_0013PM,DP03_0014E,DP03_0014M,DP03_0014PE,DP03_0014PM,DP03_0015E,DP03_0015M,DP03_0015PE,DP03_0015PM,DP03_0016E,DP03_0016M,DP03_0016PE,DP03_0016PM,DP03_0017E,DP03_0017M,DP03_0017PE,DP03_0017PM,DP03_0018E,DP03_0018M,DP03_0018PE,DP03_0018PM,DP03_0019E,DP03_0019M,DP03_0019PE,DP03_0019PM,DP03_0020E,DP03_0020M,DP03_0020PE,DP03_0020PM,DP03_0021E,DP03_0021M,DP03_0021PE,DP03_0021PM,DP03_0022E,DP03_0022M,DP03_0022PE,DP03_0022PM,DP03_0023E,DP03_0023M,DP03_0023PE,DP03_0023PM,DP03_0024E,DP03_0024M,DP03_0024PE,DP03_0024PM,DP03_0025E,DP03_0025M,DP03_0025PE,DP03_0025PM,DP03_0026E,DP03_0026M,DP03_0026PE,DP03_0026PM,DP03_0027E,DP03_0027M,DP03_0027PE,DP03_0027PM,DP03_0028E,DP03_0028M,DP03_0028PE,DP03_0028PM,DP03_0029E,DP03_0029M,DP03_0029PE,DP03_0029PM,DP03_0030E,DP03_0030M,DP03_0030PE,DP03_0030PM,DP03_0031E,DP03_0031M,DP03_0031PE,DP03_0031PM,DP03_0032E,DP03_0032M,DP03_0032PE,DP03_0032PM,DP03_0033E,DP03_0033M,DP03_0033PE,DP03_0033PM,DP03_0034E,DP03_0034M,DP03_0034PE,DP03_0034PM,DP03_0035E,DP03_0035M,DP03_0035PE,DP03_0035PM,DP03_0036E,DP03_0036M,DP03_0036PE,DP03_0036PM,DP03_0037E,DP03_0037M,DP03_0037PE,DP03_0037PM,DP03_0038E,DP03_0038M,DP03_0038PE,DP03_0038PM,DP03_0039E,DP03_0039M,DP03_0039PE,DP03_0039PM,DP03_0040E,DP03_0040M,DP03_0040PE,DP03_0040PM,DP03_0041E,DP03_0041M,DP03_0041PE,DP03_0041PM,DP03_0042E,DP03_0042M,DP03_0042PE,DP03_0042PM,DP03_0043E,DP03_0043M,DP03_0043PE,DP03_0043PM,DP03_0044E,DP03_0044M,DP03_0044PE,DP03_0044PM,DP03_0045E,DP03_0045M,DP03_0045PE,DP03_0045PM,DP03_0046E,DP03_0046M,DP03_0046PE,DP03_0046PM,DP03_0047E,DP03_0047M,DP03_0047PE,DP03_0047PM,DP03_0048E,DP03_0048M,DP03_0048PE,DP03_0048PM,DP03_0049E,DP03_0049M,DP03_0049PE,DP03_0049PM,DP03_0050E,DP03_0050M,DP03_0050PE,DP03_0050PM,DP03_0051E,DP03_0051M,DP03_0051PE,DP03_0051PM,DP03_0052E,DP03_0052M,DP03_0052PE,DP03_0052PM,DP03_0053E,DP03_0053M,DP03_0053PE,DP03_0053PM,DP03_0054E,DP03_0054M,DP03_0054PE,DP03_0054PM,DP03_0055E,DP03_0055M,DP03_0055PE,DP03_0055PM,DP03_0056E,DP03_0056M,DP03_0056PE,DP03_0056PM,DP03_0057E,DP03_0057M,DP03_0057PE,DP03_0057PM,DP03_0058E,DP03_0058M,DP03_0058PE,DP03_0058PM,DP03_0059E,DP03_0059M,DP03_0059PE,DP03_0059PM,DP03_0060E,DP03_0060M,DP03_0060PE,DP03_0060PM,DP03_0061E,DP03_0061M,DP03_0061PE,DP03_0061PM,DP03_0062E,DP03_0062M,DP03_0062PE,DP03_0062PM,DP03_0063E,DP03_0063M,DP03_0063PE,DP03_0063PM,DP03_0064E,DP03_0064M,DP03_0064PE,DP03_0064PM,DP03_0065E,DP03_0065M,DP03_0065PE,DP03_0065PM,DP03_0066E,DP03_0066M,DP03_0066PE,DP03_0066PM,DP03_0067E,DP03_0067M,DP03_0067PE,DP03_0067PM,DP03_0068E,DP03_0068M,DP03_0068PE,DP03_0068PM,DP03_0069E,DP03_0069M,DP03_0069PE,DP03_0069PM,DP03_0070E,DP03_0070M,DP03_0070PE,DP03_0070PM,DP03_0071E,DP03_0071M,DP03_0071PE,DP03_0071PM,DP03_0072E,DP03_0072M,DP03_0072PE,DP03_0072PM,DP03_0073E,DP03_0073M,DP03_0073PE,DP03_0073PM,DP03_0074E,DP03_0074M,DP03_0074PE,DP03_0074PM,DP03_0075E,DP03_0075M,DP03_0075PE,DP03_0075PM,DP03_0076E,DP03_0076M,DP03_0076PE,DP03_0076PM,DP03_0077E,DP03_0077M,DP03_0077PE,DP03_0077PM,DP03_0078E,DP03_0078M,DP03_0078PE,DP03_0078PM,DP03_0079E,DP03_0079M,DP03_0079PE,DP03_0079PM,DP03_0080E,DP03_0080M,DP03_0080PE,DP03_0080PM,DP03_0081E,DP03_0081M,DP03_0081PE,DP03_0081PM,DP03_0082E,DP03_0082M,DP03_0082PE,DP03_0082PM,DP03_0083E,DP03_0083M,DP03_0083PE,DP03_0083PM,DP03_0084E,DP03_0084M,DP03_0084PE,DP03_0084PM,DP03_0085E,DP03_0085M,DP03_0085PE,DP03_0085PM,DP03_0086E,DP03_0086M,DP03_0086PE,DP03_0086PM,DP03_0087E,DP03_0087M,DP0

In [3]:
headers = pd.read_csv('s3://bleeding-hearts/rawdata/acs_headers.csv')
headers

Unnamed: 0                                 for  \
0          label        Census API FIPS 'for' clause   
1        concept  Census API Geography Specification   
2  predicateType                            fips-for   
3          group                                 NaN   
4          limit                                   0   
5  predicateOnly                                True   
6     attributes                                 NaN   
7       required                                 NaN   

                                   in  \
0         Census API FIPS 'in' clause   
1  Census API Geography Specification   
2                             fips-in   
3                                 NaN   
4                                   0   
5                                True   
6                                 NaN   
7                                 NaN   

                                        ucgid  \
0  Uniform Census Geography Identifier clause   
1          Census API Geography Specification   
2                                       ucgid   
3                                         NaN   
4                                           0   
5                                        True   
6                                         NaN   
7                                         NaN   

                                          DP02_0126E  \
0       Estimate!!ANCESTRY!!Total population!!Danish   
1  SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...   
2                                                int   
3                                               DP02   
4                                                  0   
5                                                NaN   
6                 DP02_0126EA,DP02_0126M,DP02_0126MA   
7                                                NaN   

                                         DP05_0050PE  \
0  Percent Estimate!!RACE!!Total population!!One ...   
1              ACS DEMOGRAPHIC AND HOUSING ESTIMATES   
2                                              float   
3                                               DP05   
4                                                  0   
5                                                NaN   
6              DP05_0050PEA,DP05_0050PM,DP05_0050PMA   
7                                                NaN   

                                          DP04_0047E  \
0  Estimate!!HOUSING TENURE!!Occupied housing uni...   
1                   SELECTED HOUSING CHARACTERISTICS   
2                                                int   
3                                               DP04   
4                                                  0   
5                                                NaN   
6                 DP04_0047EA,DP04_0047M,DP04_0047MA   
7                                                NaN   

                                       DP02PR_0083PE  \
0  Percent Estimate!!RESIDENCE 1 YEAR AGO!!Popula...   
1     SELECTED SOCIAL CHARACTERISTICS IN PUERTO RICO   
2                                              float   
3                                             DP02PR   
4                                                  0   
5                                                NaN   
6        DP02PR_0083PEA,DP02PR_0083PM,DP02PR_0083PMA   
7                                                NaN   

                                       DP02PR_0106PE  \
0  Percent Estimate!!WORLD REGION OF BIRTH OF FOR...   
1     SELECTED SOCIAL CHARACTERISTICS IN PUERTO RICO   
2                                              float   
3                                             DP02PR   
4                                                  0   
5                                                NaN   
6        DP02PR_0106PEA,DP02PR_0106PM,DP02PR_0106PMA   
7                                                NaN   

                                          DP04_0046E  \
0  Estimate!!HOUSING TENURE!!Occupied housing uni...   
1                   SELECTED HOUSING CHARACTERISTICS

Turn headers data into dictionary that we can use to rename the columns. Start by transposing the header data and drop all columns except the DP descriptor and the label.

In [4]:
headers=headers.T
headers

,0,1,2,3,4,5,6,7
Unnamed: 0,label,concept,predicateType,group,limit,predicateOnly,attributes,required
for,Census API FIPS 'for' clause,Census API Geography Specification,fips-for,NaN,0,True,NaN,NaN
in,Census API FIPS 'in' clause,Census API Geography Specification,fips-in,NaN,0,True,NaN,NaN
ucgid,Uniform Census Geography Identifier clause,Census API Geography Specification,ucgid,NaN,0,True,NaN,NaN
DP02_0126E,Estimate!!ANCESTRY!!Total population!!Danish,SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...,int,DP02,0,NaN,"DP02_0126EA,DP02_0126M,DP02_0126MA",NaN
...,...,...,...,...,...,...,...,...
DP02PR_0001PE,Percent Estimate!!HOUSEHOLDS BY TYPE!!Total ho...,SELECTED SOCIAL CHARACTERISTICS IN PUERTO RICO,int,DP02PR,0,NaN,"DP02PR_0001PEA,DP02PR_0001PM,DP02PR_0001PMA",NaN
DP03_0039PE,Percent Estimate!!INDUSTRY!!Civilian employed ...,SELECTED ECONOMIC CHARACTERISTICS,float,DP03,0,NaN,"DP03_0039PEA,DP03_0039PM,DP03_0039PMA",NaN
DP02_0098E,Estimate!!YEAR OF ENTRY!!Population born outsi...,SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...,int,DP02,0,NaN,"DP02_0098EA,DP02_0098M,DP02_0098MA",NaN
DP04_0095PE,Percent Estimate!!SELECTED MONTHLY OWNER COSTS...,SELECTED HOUSING CHARACTERISTICS,float,DP04,0,NaN,"DP04_0095PEA,DP04_0095PM,DP04_0095PMA",NaN


In [5]:
headers1=headers.drop(columns=[1,2,3,4,5,6,7])
headers1

,0
Unnamed: 0,label
for,Census API FIPS 'for' clause
in,Census API FIPS 'in' clause
ucgid,Uniform Census Geography Identifier clause
DP02_0126E,Estimate!!ANCESTRY!!Total population!!Danish
...,...
DP02PR_0001PE,Percent Estimate!!HOUSEHOLDS BY TYPE!!Total ho...
DP03_0039PE,Percent Estimate!!INDUSTRY!!Civilian employed ...
DP02_0098E,Estimate!!YEAR OF ENTRY!!Population born outsi...
DP04_0095PE,Percent Estimate!!SELECTED MONTHLY OWNER COSTS...


Turn the headers dataframe into a dictionary

In [6]:
headers_d=headers1.to_dict()
headers_d=headers_d[0]
headers_d

{'Unnamed: 0': 'label',
 'for': "Census API FIPS 'for' clause",
 'in': "Census API FIPS 'in' clause",
 'ucgid': 'Uniform Census Geography Identifier clause',
 'DP02_0126E': 'Estimate!!ANCESTRY!!Total population!!Danish',
 'DP05_0050PE': 'Percent Estimate!!RACE!!Total population!!One race!!Asian!!Vietnamese',
 'DP04_0047E': 'Estimate!!HOUSING TENURE!!Occupied housing units!!Renter-occupied',
 'DP02PR_0083PE': 'Percent Estimate!!RESIDENCE 1 YEAR AGO!!Population 1 year and over!!Different house in Puerto Rico or the U.S.!!In Puerto Rico!!Different municipio',
 'DP02PR_0106PE': 'Percent Estimate!!WORLD REGION OF BIRTH OF FOREIGN BORN!!Foreign-born population, excluding population born at sea!!Africa',
 'DP04_0046E': 'Estimate!!HOUSING TENURE!!Occupied housing units!!Owner-occupied',
 'DP04_0016PE': 'Percent Estimate!!YEAR STRUCTURE BUILT!!Total housing units',
 'DP02_0125E': 'Estimate!!ANCESTRY!!Total population!!Czech',
 'DP02_0054PE': 'Percent Estimate!!SCHOOL ENROLLMENT!!Population 3 ye

Replace the !! with _.

In [7]:
for k,v in headers_d.items():
    headers_d[k]=v.replace('!!','_')
headers_d

{'Unnamed: 0': 'label',
 'for': "Census API FIPS 'for' clause",
 'in': "Census API FIPS 'in' clause",
 'ucgid': 'Uniform Census Geography Identifier clause',
 'DP02_0126E': 'Estimate_ANCESTRY_Total population_Danish',
 'DP05_0050PE': 'Percent Estimate_RACE_Total population_One race_Asian_Vietnamese',
 'DP04_0047E': 'Estimate_HOUSING TENURE_Occupied housing units_Renter-occupied',
 'DP02PR_0083PE': 'Percent Estimate_RESIDENCE 1 YEAR AGO_Population 1 year and over_Different house in Puerto Rico or the U.S._In Puerto Rico_Different municipio',
 'DP02PR_0106PE': 'Percent Estimate_WORLD REGION OF BIRTH OF FOREIGN BORN_Foreign-born population, excluding population born at sea_Africa',
 'DP04_0046E': 'Estimate_HOUSING TENURE_Occupied housing units_Owner-occupied',
 'DP04_0016PE': 'Percent Estimate_YEAR STRUCTURE BUILT_Total housing units',
 'DP02_0125E': 'Estimate_ANCESTRY_Total population_Czech',
 'DP02_0054PE': 'Percent Estimate_SCHOOL ENROLLMENT_Population 3 years and over enrolled in scho

Creating a list of all the column names in the ACS data that we will use cycle through to compare to the headers dictionary.

In [8]:
cols=list(acs)
cols

['Unnamed: 0',
 'GEO_ID',
 'DP03_0001E',
 'DP03_0001M',
 'DP03_0001PE',
 'DP03_0001PM',
 'DP03_0002E',
 'DP03_0002M',
 'DP03_0002PE',
 'DP03_0002PM',
 'DP03_0003E',
 'DP03_0003M',
 'DP03_0003PE',
 'DP03_0003PM',
 'DP03_0004E',
 'DP03_0004M',
 'DP03_0004PE',
 'DP03_0004PM',
 'DP03_0005E',
 'DP03_0005M',
 'DP03_0005PE',
 'DP03_0005PM',
 'DP03_0006E',
 'DP03_0006M',
 'DP03_0006PE',
 'DP03_0006PM',
 'DP03_0007E',
 'DP03_0007M',
 'DP03_0007PE',
 'DP03_0007PM',
 'DP03_0008E',
 'DP03_0008M',
 'DP03_0008PE',
 'DP03_0008PM',
 'DP03_0009E',
 'DP03_0009M',
 'DP03_0009PE',
 'DP03_0009PM',
 'DP03_0010E',
 'DP03_0010M',
 'DP03_0010PE',
 'DP03_0010PM',
 'DP03_0011E',
 'DP03_0011M',
 'DP03_0011PE',
 'DP03_0011PM',
 'DP03_0012E',
 'DP03_0012M',
 'DP03_0012PE',
 'DP03_0012PM',
 'DP03_0013E',
 'DP03_0013M',
 'DP03_0013PE',
 'DP03_0013PM',
 'DP03_0014E',
 'DP03_0014M',
 'DP03_0014PE',
 'DP03_0014PM',
 'DP03_0015E',
 'DP03_0015M',
 'DP03_0015PE',
 'DP03_0015PM',
 'DP03_0016E',
 'DP03_0016M',
 'DP03_0016PE'

In [9]:
len(cols)

1100

Removing attributes from the headers. See https://api.census.gov/data/2018/acs/acs1/profile/variables.html that the actual variables end with "E" or "PE". The attributes end with a variety of letter combinations. At the lowest level, the attributes can be removed by searching for "EA" and "M" to remove them.

In [10]:
cols=[i for i in cols if "EA" not in i]
len(cols)

826

In [11]:
cols=[i for i in cols if "M" not in i]
len(cols)

277

Creating another dictionary. Looping through the list of column names from the ACS dataset and if they are in the headers dictionary, placing them in the new dictionary that will contain only headers in the ACS dataset.

In [12]:
dp03={}
for k, v in headers_d.items():
    if k in cols:
        dp03[k]=v
dp03

{'Unnamed: 0': 'label',
 'DP03_0048PE': 'Percent Estimate_CLASS OF WORKER_Civilian employed population 16 years and over_Government workers',
 'DP03_0022PE': 'Percent Estimate_COMMUTING TO WORK_Workers 16 years and over_Walked',
 'DP03_0111PE': 'Percent Estimate_HEALTH INSURANCE COVERAGE_Civilian noninstitutionalized population_Civilian noninstitutionalized population 19 to 64 years_In labor force_Unemployed_With health insurance coverage_With private health insurance',
 'DP03_0109E': 'Estimate_HEALTH INSURANCE COVERAGE_Civilian noninstitutionalized population_Civilian noninstitutionalized population 19 to 64 years_In labor force_Unemployed',
 'DP03_0035PE': 'Percent Estimate_INDUSTRY_Civilian employed population 16 years and over_Manufacturing',
 'DP03_0066PE': 'Percent Estimate_INCOME AND BENEFITS (IN 2018 INFLATION-ADJUSTED DOLLARS)_Total households_With Social Security',
 'DP03_0107E': 'Estimate_HEALTH INSURANCE COVERAGE_Civilian noninstitutionalized population_Civilian noninstitut

In [13]:
len(dp03)

276

Removing the attributes.

In [14]:
acs=acs[cols]
acs.columns

Index(['Unnamed: 0', 'GEO_ID', 'DP03_0001E', 'DP03_0001PE', 'DP03_0002E',
       'DP03_0002PE', 'DP03_0003E', 'DP03_0003PE', 'DP03_0004E', 'DP03_0004PE',
       ...
       'DP03_0133PE', 'DP03_0134E', 'DP03_0134PE', 'DP03_0135E', 'DP03_0135PE',
       'DP03_0136E', 'DP03_0136PE', 'DP03_0137E', 'DP03_0137PE',
       'metropolitan statistical area/micropolitan statistical area'],
      dtype='object', length=277)

Renaming the columns.

In [15]:
acs.rename(columns=dp03,inplace=True)
acs.head()

,label,Geography,Estimate_EMPLOYMENT STATUS_Population 16 years and over,Percent Estimate_EMPLOYMENT STATUS_Population 16 years and over,Estimate_EMPLOYMENT STATUS_Population 16 years and over_In labor force,Percent Estimate_EMPLOYMENT STATUS_Population 16 years and over_In labor force,Estimate_EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force,Percent Estimate_EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force,Estimate_EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force_Employed,Percent Estimate_EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force_Employed,Estimate_EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force_Unemployed,Percent Estimate_EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force_Unemployed,Estimate_EMPLOYMENT STATUS_Population 16 years and over_In labor force_Armed Forces,Percent Estimate_EMPLOYMENT STATUS_Population 16 years and over_In labor force_Armed Forces,Estimate_EMPLOYMENT STATUS_Population 16 years and over_Not in labor force,Percent Estimate_EMPLOYMENT STATUS_Population 16 years and over_Not in labor force,Estimate_EMPLOYMENT STATUS_Civilian labor force,Percent Estimate_EMPLOYMENT STATUS_Civilian labor force,Estimate_EMPLOYMENT STATUS_Civilian labor force_Unemployment Rate,Percent Estimate_EMPLOYMENT STATUS_Civilian labor force_Unemployment Rate,Estimate_EMPLOYMENT STATUS_Females 16 years and over,Percent Estimate_EMPLOYMENT STATUS_Females 16 years and over,Estimate_EMPLOYMENT STATUS_Females 16 years and over_In labor force,Percent Estimate_EMPLOYMENT STATUS_Females 16 years and over_In labor force,Estimate_EMPLOYMENT STATUS_Females 16 years and over_In labor force_Civilian labor force,Percent Estimate_EMPLOYMENT STATUS_Females 16 years and over_In labor force_Civilian labor force,Estimate_EMPLOYMENT STATUS_Females 16 years and over_In labor force_Civilian labor force_Employed,Percent Estimate_EMPLOYMENT STATUS_Females 16 years and over_In labor force_Civilian labor force_Employed,Estimate_EMPLOYMENT STATUS_Own children of the householder under 6 years,Percent Estimate_EMPLOYMENT STATUS_Own children of the householder under 6 years,Estimate_EMPLOYMENT STATUS_Own children of the householder under 6 years_All parents in family in labor force,Percent Estimate_EMPLOYMENT STATUS_Own children of the householder under 6 years_All parents in family in labor force,Estimate_EMPLOYMENT STATUS_Own children of the householder 6 to 17 years,Percent Estimate_EMPLOYMENT STATUS_Own children of the householder 6 to 17 years,Estimate_EMPLOYMENT STATUS_Own children of the householder 6 to 17 years_All parents in family in labor force,Percent Estimate_EMPLOYMENT STATUS_Own children of the householder 6 to 17 years_All parents in family in labor force,Estimate_COMMUTING TO WORK_Workers 16 years and over,Percent Estimate_COMMUTING TO WORK_Workers 16 years and over,"Estimate_COMMUTING TO WORK_Workers 16 years and over_Car, truck, or van -- drove alone","Percent Estimate_COMMUTING TO WORK_Workers 16 years and over_Car, truck, or van -- drove alone","Estimate_COMMUTING TO WORK_Workers 16 years and over_Car, truck, or van -- carpooled","Percent Estimate_COMMUTING TO WORK_Workers 16 years and over_Car, truck, or van -- carpooled",Estimate_COMMUTING TO WORK_Workers 16 years and over_Public transportation (excluding taxicab),Percent Estimate_COMMUTING TO WORK_Workers 16 years and over_Public transportation (excluding taxicab),Estimate_COMMUTING TO WORK_Workers 16 years and over_Walked,Percent Estimate_COMMUTING TO WORK_Workers 16 years and over_Walked,Estimate_COMMUTING TO WORK_Workers 16 years and over_Other means,Percent Estimate_COMMUTING TO WORK_Workers 16 years and over_Other means,Estimate_COMMUTING TO WORK_Workers 16 years and over_Worked at home,Percent Estimate_COMMUTING TO WORK_Workers 16 years and over_Worked at home,Estimate_COMMUTIN

Checking the acs column types.

In [16]:
acs.dtypes

label                                                                                                                                                         int64
Geography                                                                                                                                                    object
Estimate_EMPLOYMENT STATUS_Population 16 years and over                                                                                                       int64
Percent Estimate_EMPLOYMENT STATUS_Population 16 years and over                                                                                               int64
Estimate_EMPLOYMENT STATUS_Population 16 years and over_In labor force                                                                                        int64
                                                                                                                                                             ...   
Estimate_PERCENT

Geography variable in the COI data does not have the letters. Removing letters in this dataset, casting the Geography variable to integer, and changing the column name to geoid to match COI data so we can merge.

In [21]:
#try:
#    acs['Geography'] = acs['Geography'].str.replace('M', '')
#    acs['Geography'] = acs['Geography'].str.replace('US', '').astype(int)
#except AttributeError:
#    pass
#acs.rename(columns={'Geography': 'geoid'},inplace=True)
#acs.head()


,label,geoid,Estimate_EMPLOYMENT STATUS_Population 16 years and over,Percent Estimate_EMPLOYMENT STATUS_Population 16 years and over,Estimate_EMPLOYMENT STATUS_Population 16 years and over_In labor force,Percent Estimate_EMPLOYMENT STATUS_Population 16 years and over_In labor force,Estimate_EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force,Percent Estimate_EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force,Estimate_EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force_Employed,Percent Estimate_EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force_Employed,Estimate_EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force_Unemployed,Percent Estimate_EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force_Unemployed,Estimate_EMPLOYMENT STATUS_Population 16 years and over_In labor force_Armed Forces,Percent Estimate_EMPLOYMENT STATUS_Population 16 years and over_In labor force_Armed Forces,Estimate_EMPLOYMENT STATUS_Population 16 years and over_Not in labor force,Percent Estimate_EMPLOYMENT STATUS_Population 16 years and over_Not in labor force,Estimate_EMPLOYMENT STATUS_Civilian labor force,Percent Estimate_EMPLOYMENT STATUS_Civilian labor force,Estimate_EMPLOYMENT STATUS_Civilian labor force_Unemployment Rate,Percent Estimate_EMPLOYMENT STATUS_Civilian labor force_Unemployment Rate,Estimate_EMPLOYMENT STATUS_Females 16 years and over,Percent Estimate_EMPLOYMENT STATUS_Females 16 years and over,Estimate_EMPLOYMENT STATUS_Females 16 years and over_In labor force,Percent Estimate_EMPLOYMENT STATUS_Females 16 years and over_In labor force,Estimate_EMPLOYMENT STATUS_Females 16 years and over_In labor force_Civilian labor force,Percent Estimate_EMPLOYMENT STATUS_Females 16 years and over_In labor force_Civilian labor force,Estimate_EMPLOYMENT STATUS_Females 16 years and over_In labor force_Civilian labor force_Employed,Percent Estimate_EMPLOYMENT STATUS_Females 16 years and over_In labor force_Civilian labor force_Employed,Estimate_EMPLOYMENT STATUS_Own children of the householder under 6 years,Percent Estimate_EMPLOYMENT STATUS_Own children of the householder under 6 years,Estimate_EMPLOYMENT STATUS_Own children of the householder under 6 years_All parents in family in labor force,Percent Estimate_EMPLOYMENT STATUS_Own children of the householder under 6 years_All parents in family in labor force,Estimate_EMPLOYMENT STATUS_Own children of the householder 6 to 17 years,Percent Estimate_EMPLOYMENT STATUS_Own children of the householder 6 to 17 years,Estimate_EMPLOYMENT STATUS_Own children of the householder 6 to 17 years_All parents in family in labor force,Percent Estimate_EMPLOYMENT STATUS_Own children of the householder 6 to 17 years_All parents in family in labor force,Estimate_COMMUTING TO WORK_Workers 16 years and over,Percent Estimate_COMMUTING TO WORK_Workers 16 years and over,"Estimate_COMMUTING TO WORK_Workers 16 years and over_Car, truck, or van -- drove alone","Percent Estimate_COMMUTING TO WORK_Workers 16 years and over_Car, truck, or van -- drove alone","Estimate_COMMUTING TO WORK_Workers 16 years and over_Car, truck, or van -- carpooled","Percent Estimate_COMMUTING TO WORK_Workers 16 years and over_Car, truck, or van -- carpooled",Estimate_COMMUTING TO WORK_Workers 16 years and over_Public transportation (excluding taxicab),Percent Estimate_COMMUTING TO WORK_Workers 16 years and over_Public transportation (excluding taxicab),Estimate_COMMUTING TO WORK_Workers 16 years and over_Walked,Percent Estimate_COMMUTING TO WORK_Workers 16 years and over_Walked,Estimate_COMMUTING TO WORK_Workers 16 years and over_Other means,Percent Estimate_COMMUTING TO WORK_Workers 16 years and over_Other means,Estimate_COMMUTING TO WORK_Workers 16 years and over_Worked at home,Percent Estimate_COMMUTING TO WORK_Workers 16 years and over_Worked at home,Estimate_COMMUTING TO

Uploading to S3.

In [ ]:
with open('/Volumes/Key/aws_keys.json') as f:
  creds = json.load(f)

In [ ]:
def s3upload(name,df):
    x = df.to_csv(None).encode()
    fs = s3fs.S3FileSystem(key=creds['AWSAccessKeyId'], secret=creds['AWSSecretKey'])
    with fs.open('s3://bleeding-hearts/workingdata/' + name + '.csv', 'wb') as f:
        f.write(x)
    print('saved')

In [ ]:
#s3upload('acs',acs)

# COI 

In [12]:
def readcoi(file):
    df = pd.read_csv(file)
    df = df[df.msaid15.notnull()]
    df['msaid15'] = df['msaid15'].astype(int)
    return df

In [ ]:
coi_index = readcoi('s3://bleeding-hearts/rawdata/coi_index.csv')
coi_index.head()

In [13]:
coi_indicator = readcoi('s3://bleeding-hearts/rawdata/coi_rawindicator.csv')
coi_indicator.head()

<ipython-input-12-2b1c09378f57>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['msaid15'] = df['msaid15'].astype(int)


## COI Headers

In [ ]:
cols = ['Column','Type','Label','Description']
def read_headers(file,pg,table):
    table = tabula.read_pdf(file,
                            pages=pg,
                           multiple_tables=True)
    head=table[0]
    if len(head.columns) == 5:
        head.drop(head.columns[-1],axis=1,inplace=True)
    elif len(head.columns) == 6:
        head.drop(head.columns[-2],axis=1,inplace=True)
    head.columns=cols
    return(head)

In [43]:
index_h2 = read_headers('/Users/sarahmartin/Documents/GT/COI_index_headers.pdf',
                        3,
                        0)
index_h2

ValueError: Length mismatch: Expected axis has 5 elements, new values have 4 elements

In [27]:
#file = "s3://bleeding-hearts/rawdata/COI_index_headers.pdf"
table = tabula.read_pdf('/Users/sarahmartin/Documents/GT/COI_index_headers.pdf',
                        pages=2,
                       multiple_tables=True)
c_head1=table[2]
c_head1

,0,1,2,3
0,Column,Type,Label,Description
1,geoid,text,2010 census tract FIPS codes,NaN
2,year,numeric,Year of observation (2010 or,NaN
3,NaN,NaN,2015),NaN
4,in100,numeric,In one of the 100 largest metro,This variable is equal to 1 if the
5,NaN,NaN,areas,census tract is located in one of
6,NaN,NaN,NaN,the 100 largest metro areas (as
7,NaN,NaN,NaN,"of 2015), zero if located in a"
8,NaN,NaN,NaN,metro- or micropolitan area
9,NaN,NaN,NaN,that does not belong to the 100


In [26]:
c_head1 = c_head1[c_head1['Column'].notna()]
cols=list(c_head1)
c_head1

KeyError: 'Column'

In [69]:
table = tabula.read_pdf(file,
                        pages=3)
c_head2=table[0]
c_head2

,Unnamed: 0,countyfips,text,County FIPS Code,Unnamed: 1,Unnamed: 2
0,statefips,text,State FIPS Code,NaN,NaN,NaN
1,stateusps,text,State USPS Code,NaN,NaN,NaN
2,pop,numeric,"Number of children aged 0-17,\rACS (2008-12 fo...",NaN,NaN,NaN
3,z_ED_nat,numeric,"Z-scores, education domain,\rnationally-normed",Weighted average of education\rdomain componen...,NaN,NaN
4,z_HE_nat,numeric,"Z-scores, health and\renvironment domain,\rnat...",Weighted average of health and\renvironment do...,NaN,NaN
5,z_SE_nat,numeric,"Z-scores, social and economic\rdomain, nationa...",Weighted average of social and\reconomic domai...,NaN,NaN
6,z_COI_nat,numeric,"Z-scores, overall COI,\rnationally-normed",Weighted average of three\rdomain averaged z-s...,NaN,NaN
7,c5_ED_nat,text,"Child Opportunity Levels,\reducation domain, n...",Nationally-normed Child\rOpportunity Levels (f...,NaN,NaN
8,c5_HE_nat,text,"Child Opportunity Levels,\rhealth and environm...",Nationally-normed Child\rOpportunity Levels (f...,NaN,NaN
9,c5_SE_nat,text,"Child Opportunity Levels, social\rand economic...",Nationally-normed Child\rOpportunity Levels (f...,NaN,NaN


In [70]:
def clean(df,dropcols):
    df.drop(columns=dropcols,inplace=True)
    df.columns = cols
    df['Label'] = df['Label'].str.replace('\r',' ')
    return df

In [71]:
c_head2=clean(c_head2,['Unnamed: 1','Unnamed: 2'])
c_head2

,Column,Type,Label,Description
0,statefips,text,State FIPS Code,NaN
1,stateusps,text,State USPS Code,NaN
2,pop,numeric,"Number of children aged 0-17, ACS (2008-12 for...",NaN
3,z_ED_nat,numeric,"Z-scores, education domain, nationally-normed",Weighted average of education\rdomain componen...
4,z_HE_nat,numeric,"Z-scores, health and environment domain, natio...",Weighted average of health and\renvironment do...
5,z_SE_nat,numeric,"Z-scores, social and economic domain, national...",Weighted average of social and\reconomic domai...
6,z_COI_nat,numeric,"Z-scores, overall COI, nationally-normed",Weighted average of three\rdomain averaged z-s...
7,c5_ED_nat,text,"Child Opportunity Levels, education domain, na...",Nationally-normed Child\rOpportunity Levels (f...
8,c5_HE_nat,text,"Child Opportunity Levels, health and environme...",Nationally-normed Child\rOpportunity Levels (f...
9,c5_SE_nat,text,"Child Opportunity Levels, social and economic ...",Nationally-normed Child\rOpportunity Levels (f...


In [72]:
table = tabula.read_pdf(file,
                        pages=4)
c_head3=table[0]
c_head3

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,economic domain.,Unnamed: 4
0,r_COI_nat,numeric,"Child Opportunity Scores,\roverall COI, nation...",Nationally-normed Child\rOpportunity Scores (f...,NaN,NaN
1,c5_ED_stt,text,"Child Opportunity Levels,\reducation domain, s...","State-normed Child\rOpportunity Levels (from ""...",NaN,NaN
2,c5_HE_stt,text,"Child Opportunity Levels,\rhealth and environm...","State-normed Child\rOpportunity Levels (from ""...",NaN,NaN
3,c5_SE_stt,text,"Child Opportunity Levels, social\rand economic...","State-normed Child\rOpportunity Levels (from ""...",NaN,NaN
4,c5_COI_stt,text,"Child Opportunity Levels,\roverall COI, state-...","State-normed Child\rOpportunity Levels (from ""...",NaN,NaN
5,r_ED_stt,numeric,"Child Opportunity Scores,\reducation domain, s...",State-normed Child\rOpportunity Scores (from 1...,NaN,NaN
6,r_HE_stt,numeric,"Child Opportunity Scores,\rhealth and environm...",State-normed Child\rOpportunity Scores (from 1...,NaN,NaN
7,r_SE_stt,numeric,"Child Opportunity Scores,\rsocial and economic...",State-normed Child\rOpportunity Scores (from 1...,NaN,NaN
8,r_COI_stt,numeric,"Child Opportunity Scores,\roverall COI, state-...",State-normed Child\rOpportunity Scores (from 1...,NaN,NaN
9,c5_ED_met,text,"Child Opportunity Levels,\reducation domain, m...","Metro-normed Child\rOpportunity Levels (from ""...",NaN,NaN


In [73]:
c_head3=clean(c_head3,['economic domain.','Unnamed: 4'])
c_head3

,Column,Type,Label,Description
0,r_COI_nat,numeric,"Child Opportunity Scores, overall COI, nationa...",Nationally-normed Child\rOpportunity Scores (f...
1,c5_ED_stt,text,"Child Opportunity Levels, education domain, st...","State-normed Child\rOpportunity Levels (from ""..."
2,c5_HE_stt,text,"Child Opportunity Levels, health and environme...","State-normed Child\rOpportunity Levels (from ""..."
3,c5_SE_stt,text,"Child Opportunity Levels, social and economic ...","State-normed Child\rOpportunity Levels (from ""..."
4,c5_COI_stt,text,"Child Opportunity Levels, overall COI, state-n...","State-normed Child\rOpportunity Levels (from ""..."
5,r_ED_stt,numeric,"Child Opportunity Scores, education domain, st...",State-normed Child\rOpportunity Scores (from 1...
6,r_HE_stt,numeric,"Child Opportunity Scores, health and environme...",State-normed Child\rOpportunity Scores (from 1...
7,r_SE_stt,numeric,"Child Opportunity Scores, social and economic ...",State-normed Child\rOpportunity Scores (from 1...
8,r_COI_stt,numeric,"Child Opportunity Scores, overall COI, state-n...",State-normed Child\rOpportunity Scores (from 1...
9,c5_ED_met,text,"Child Opportunity Levels, education domain, me...","Metro-normed Child\rOpportunity Levels (from ""..."


In [85]:
table = tabula.read_pdf(file,
                        pages=5,
                       multiple_tables=True)
c_head4=table[0]
c_head4

,"social and economic domain.\rc5_COI_mettextChild Opportunity Levels,Metro-normed Child\roverall COI, metro-normedOpportunity Levels (from ""very\rlow"" to ""very high"") for the\roverall COI.\rr_ED_metnumericChild Opportunity Scores,Metro-normed Child\reducation domain, metro-Opportunity Scores (from 1 to\rnormed100) for the education domain.\rr_HE_metnumericChild Opportunity Scores,Metro-normed Child\rhealth and environmentOpportunity Scores (from 1 to\rdomain, metro-normed100) for the health and\renvironment domain.\rr_SE_metnumericChild Opportunity Scores,Metro-normed Child\rsocial and economic domain,Opportunity Scores (from 1 to\rmetro-normed100) for the social and\reconomic domain.\rr_COI_metnumericChild Opportunity Scores,Metro-normed Child\roverall COI, metro-normedOpportunity Scores (from 1 to\r100) for the overall COI.",Unnamed: 0,Unnamed: 1,Unnamed: 2,social and economic domain.
0,c5_COI_met,text,"Child Opportunity Levels,\roverall COI, metro-...","Metro-normed Child\rOpportunity Levels (from ""...",NaN
1,r_ED_met,numeric,"Child Opportunity Scores,\reducation domain, m...",Metro-normed Child\rOpportunity Scores (from 1...,NaN
2,r_HE_met,numeric,"Child Opportunity Scores,\rhealth and environm...",Metro-normed Child\rOpportunity Scores (from 1...,NaN
3,r_SE_met,numeric,"Child Opportunity Scores,\rsocial and economic...",Metro-normed Child\rOpportunity Scores (from 1...,NaN
4,r_COI_met,numeric,"Child Opportunity Scores,\roverall COI, metro-...",Metro-normed Child\rOpportunity Scores (from 1...,NaN
5,Get updates from\rJoin our mailing list (https...,NaN,NaN,NaN,NaN


In [86]:
c_head4=clean(c_head4,['social and economic domain.'])
c_head4.drop(c_head4.tail(1).index,inplace=True)
c_head4

,Column,Type,Label,Description
0,c5_COI_met,text,"Child Opportunity Levels, overall COI, metro-n...","Metro-normed Child\rOpportunity Levels (from ""..."
1,r_ED_met,numeric,"Child Opportunity Scores, education domain, me...",Metro-normed Child\rOpportunity Scores (from 1...
2,r_HE_met,numeric,"Child Opportunity Scores, health and environme...",Metro-normed Child\rOpportunity Scores (from 1...
3,r_SE_met,numeric,"Child Opportunity Scores, social and economic ...",Metro-normed Child\rOpportunity Scores (from 1...
4,r_COI_met,numeric,"Child Opportunity Scores, overall COI, metro-n...",Metro-normed Child\rOpportunity Scores (from 1...


In [94]:
c_head = pd.concat([c_head1,c_head2,c_head3,c_head4])
c_head.drop(columns=['Type','Description'],inplace=True)
c_head.set_index('Column',inplace=True)
c_head=c_head.T
c_head

Column,geoid,year,in100,msaid15,msaname15,statefips,stateusps,pop,z_ED_nat,z_HE_nat,z_SE_nat,z_COI_nat,c5_ED_nat,c5_HE_nat,c5_SE_nat,c5_COI_nat,r_ED_nat,r_HE_nat,r_SE_nat,r_COI_nat,c5_ED_stt,c5_HE_stt,c5_SE_stt,c5_COI_stt,r_ED_stt,r_HE_stt,r_SE_stt,r_COI_stt,c5_ED_met,c5_HE_met,c5_SE_met,c5_COI_met,r_ED_met,r_HE_met,r_SE_met,r_COI_met
Label,2010 census tract FIPS codes,Year of observation (2010 or,In one of the 100 largest metro,Metro/Micro Area (2015) FIPS,Metro/Micro Area (2015) Name,State FIPS Code,State USPS Code,"Number of children aged 0-17, ACS (2008-12 for...","Z-scores, education domain, nationally-normed","Z-scores, health and environment domain, natio...","Z-scores, social and economic domain, national...","Z-scores, overall COI, nationally-normed","Child Opportunity Levels, education domain, na...","Child Opportunity Levels, health and environme...","Child Opportunity Levels, social and economic ...","Child Opportunity Levels, overall COI, nationa...","Child Opportunity Scores, education domain, na...","Child Opportunity Scores, health and environme...","Child Opportunity Scores, social and economic ...","Child Opportunity Scores, overall COI, nationa...","Child Opportunity Levels, education domain, st...","Child Opportunity Levels, health and environme...","Child Opportunity Levels, social and economic ...","Child Opportunity Levels, overall COI, state-n...","Child Opportunity Scores, education domain, st...","Child Opportunity Scores, health and environme...","Child Opportunity Scores, social and economic ...","Child Opportunity Scores, overall COI, state-n...","Child Opportunity Levels, education domain, me...","Child Opportunity Levels, health and environme...","Child Opportunity Levels, social and economic ...","Child Opportunity Levels, overall COI, metro-n...","Child Opportunity Scores, education domain, me...","Child Opportunity Scores, health and environme...","Child Opportunity Scores, social and economic ...","Child Opportunity Scores, overall COI, metro-n..."


In [99]:
c_head_d = c_head.to_dict('records')
c_head_d = c_head_d[0]
c_head_d

{'geoid': '2010 census tract FIPS codes',
 'year': 'Year of observation (2010 or',
 'in100': 'In one of the 100 largest metro',
 'msaid15': 'Metro/Micro Area (2015) FIPS',
 'msaname15': 'Metro/Micro Area (2015) Name',
 'statefips': 'State FIPS Code',
 'stateusps': 'State USPS Code',
 'pop': 'Number of children aged 0-17, ACS (2008-12 for 2010, 2013-17 for 2015)',
 'z_ED_nat': 'Z-scores, education domain, nationally-normed',
 'z_HE_nat': 'Z-scores, health and environment domain, nationally-normed',
 'z_SE_nat': 'Z-scores, social and economic domain, nationally-normed',
 'z_COI_nat': 'Z-scores, overall COI, nationally-normed',
 'c5_ED_nat': 'Child Opportunity Levels, education domain, nationally- normed',
 'c5_HE_nat': 'Child Opportunity Levels, health and environment domain, nationally-normed',
 'c5_SE_nat': 'Child Opportunity Levels, social and economic domain, nationally-normed',
 'c5_COI_nat': 'Child Opportunity Levels, overall COI, nationally-normed',
 'r_ED_nat': 'Child Opportunity

In [100]:
coi_index.rename(columns=c_head_d,inplace=True)
coi_index

,Unnamed: 0,2010 census tract FIPS codes,Year of observation (2010 or,In one of the 100 largest metro,Metro/Micro Area (2015) FIPS,Metro/Micro Area (2015) Name,countyfips,State FIPS Code,State USPS Code,"Number of children aged 0-17, ACS (2008-12 for 2010, 2013-17 for 2015)","Z-scores, education domain, nationally-normed","Z-scores, health and environment domain, nationally-normed","Z-scores, social and economic domain, nationally-normed","Z-scores, overall COI, nationally-normed","Child Opportunity Levels, education domain, nationally- normed","Child Opportunity Levels, health and environment domain, nationally-normed","Child Opportunity Levels, social and economic domain, nationally-normed","Child Opportunity Levels, overall COI, nationally-normed","Child Opportunity Scores, education domain, nationally- normed","Child Opportunity Scores, health and environment domain, nationally-normed","Child Opportunity Scores, social and economic domain, nationally-normed","Child Opportunity Scores, overall COI, nationally-normed","Child Opportunity Levels, education domain, state- normed","Child Opportunity Levels, health and environment domain, state-normed","Child Opportunity Levels, social and economic domain, state- normed","Child Opportunity Levels, overall COI, state-normed","Child Opportunity Scores, education domain, state- normed","Child Opportunity Scores, health and environment domain, state-normed","Child Opportunity Scores, social and economic domain, state-normed","Child Opportunity Scores, overall COI, state-normed","Child Opportunity Levels, education domain, metro- normed","Child Opportunity Levels, health and environment domain, metro-normed","Child Opportunity Levels, social and economic domain, metro- normed","Child Opportunity Levels, overall COI, metro-normed","Child Opportunity Scores, education domain, metro- normed","Child Opportunity Scores, health and environment domain, metro-normed","Child Opportunity Scores, social and economic domain, metro-normed","Child Opportunity Scores, overall COI, metro-normed"
0,0,1001020100,2010,0.0,33860,"Montgomery, AL Metro Area",1001,1,AL,424,0.001198,-0.016530,0.046101,0.003759,Moderate,Very Low,Moderate,Moderate,55,20,54,49,High,Low,High,High,67,33,67,65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1001020100,2015,0.0,33860,"Montgomery, AL Metro Area",1001,1,AL,414,0.008932,0.035614,0.146657,0.021866,Moderate,High,High,High,59,66,75,71,High,Very High,Very High,Very High,71,92,83,84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1001020200,2010,0.0,33860,"Montgomery, AL Metro Area",1001,1,AL,497,0.013306,-0.022400,-0.133538,-0.016473,High,Very Low,Low,Low,62,17,23,27,High,Low,Low,Low,73,28,31,37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1001020200,2015,0.0,33860,"Montgomery, AL Metro Area",1001,1,AL,508,-0.003518,-0.012027,-0.169899,-0.021479,Moderate,Low,Very Low,Low,53,23,18,23,High,Low,Low,Low,64,39,26,32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1001020300,2010,0.0,33860,"Montgomery, AL Metro Area",1001,1,AL,878,-0.014533,-0.026710,0.102040,0.007462,Moderate,Very Low,High,Moderate,46,15,66,54,Moderate,Low,High,High,59,23,77,69,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144393,144393,56041975200,2015,0.0,21740,"Evanston, WY Micro Area",56041,56,WY,1937,-0.001343,0.045105,0.097755,0.016030,Moderate,High,High,High,54,75,65,64,Moderate,Moderate,Moderate,Moderate,57,51,45,51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
144394,144394,56041975300,2010,0.0,21740,"Evanston, WY Micro Area",56041,56,WY,2106,-0.036290,0.020205,0.072795,0.006610,Low,Moderate,Moderate,Moderate,33,49,59,53,Low,Very Low,Low,Low,22,19,36,27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
144395,144395,56041975300,2015,0.0,21740,"Evanston, WY Micro Area",56041,56,WY,2322,0.019002,0.041270,0.042496,0.011433,High,High,Moderate,Moderate,64,72,53,59,High,Moderate,Low,Low,73,45,27,35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
144396,144396

In [11]:
coi_indicator = pd.read_csv('s3://bleeding-hearts/rawdata/coi_rawindicator.csv')
coi_indicator.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146112 entries, 0 to 146111
Data columns (total 39 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Unnamed: 0   146112 non-null  int64  
 1   geoid        146112 non-null  int64  
 2   year         146112 non-null  int64  
 3   in100        135352 non-null  float64
 4   msaid15      135352 non-null  float64
 5   msaname15    135352 non-null  object 
 6   countyfips   146112 non-null  int64  
 7   statefips    146112 non-null  int64  
 8   stateusps    146112 non-null  object 
 9   pop          146112 non-null  int64  
 10  ED_APENR     144411 non-null  float64
 11  ED_ATTAIN    144823 non-null  float64
 12  ED_COLLEGE   139301 non-null  float64
 13  ED_ECENROL   141161 non-null  float64
 14  ED_HSGRAD    144142 non-null  float64
 15  ED_MATH      143647 non-null  float64
 16  ED_READING   143646 non-null  float64
 17  ED_SCHPOV    144717 non-null  float64
 18  ED_TEACHXP   144640 non-

In [30]:
file = '/Users/sarahmartin/Documents/GT/COI_indicator_headers.pdf'
table = tabula.read_pdf(file,
                        pages=5,
                       multiple_tables=True)
c_head2=table[0]
#c_head1.drop(columns=[4,5],inplace=True)
#c_head1.columns=cols
c_head2

,0,1,2,3,4
0,SE_JOBPROXnumericCommute durationPercentage wo...,SE_JOBPROX,numeric,Commute duration,Percentage workers\rcommuting more than one\rh...
1,SE_SINGLE,numeric,Single-headed households,Percentage family households\rthat are single-...,NaN
2,Get updates from\rJoin our mailing list (https...,NaN,NaN,NaN,NaN


In [40]:
def read_headers(file,pg,table):
    table = tabula.read_pdf(file,
                            pages=pg,
                           multiple_tables=True)
    head=table[0]
    if len(head.columns) == 5:
        head.drop(head.columns[-1],axis=1,inplace=True)
    elif len(head.columns) == 6:
        head.drop(head.columns[-2],axis=1,inplace=True)
    head.columns=cols
    return(head)

In [41]:
indicator1 = read_headers('/Users/sarahmartin/Documents/GT/COI_indicator_headers.pdf',
                         5,
                         0)
indicator1

,Column,Type,Label,Description
0,SE_JOBPROXnumericCommute durationPercentage wo...,SE_JOBPROX,numeric,Commute duration
1,SE_SINGLE,numeric,Single-headed households,Percentage family households\rthat are single-...
2,Get updates from\rJoin our mailing list (https...,NaN,NaN,NaN


In [ ]:
    file = '/Users/sarahmartin/Documents/GT/COI_indicator_headers.pdf'
    table = tabula.read_pdf(file,
                            pages=3,
                           multiple_tables=True)
    c_head1=table[0]
    c_head1.drop(columns=[4,5],inplace=True)
    c_head1.columns=cols
    c_head1

In [40]:
merge=acs.merge(coi_index,
               left_on='metropolitan statistical area/micropolitan statistical area',
               right_on='msaid15')
merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 125899 entries, 0 to 125898
Columns: 315 entries, label to r_COI_met
dtypes: float64(132), int64(169), object(14)
memory usage: 303.5+ MB


In [41]:
merge.head()

,label,geoid_x,Estimate_EMPLOYMENT STATUS_Population 16 years and over,Percent Estimate_EMPLOYMENT STATUS_Population 16 years and over,Estimate_EMPLOYMENT STATUS_Population 16 years and over_In labor force,Percent Estimate_EMPLOYMENT STATUS_Population 16 years and over_In labor force,Estimate_EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force,Percent Estimate_EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force,Estimate_EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force_Employed,Percent Estimate_EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force_Employed,Estimate_EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force_Unemployed,Percent Estimate_EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force_Unemployed,Estimate_EMPLOYMENT STATUS_Population 16 years and over_In labor force_Armed Forces,Percent Estimate_EMPLOYMENT STATUS_Population 16 years and over_In labor force_Armed Forces,Estimate_EMPLOYMENT STATUS_Population 16 years and over_Not in labor force,Percent Estimate_EMPLOYMENT STATUS_Population 16 years and over_Not in labor force,Estimate_EMPLOYMENT STATUS_Civilian labor force,Percent Estimate_EMPLOYMENT STATUS_Civilian labor force,Estimate_EMPLOYMENT STATUS_Civilian labor force_Unemployment Rate,Percent Estimate_EMPLOYMENT STATUS_Civilian labor force_Unemployment Rate,Estimate_EMPLOYMENT STATUS_Females 16 years and over,Percent Estimate_EMPLOYMENT STATUS_Females 16 years and over,Estimate_EMPLOYMENT STATUS_Females 16 years and over_In labor force,Percent Estimate_EMPLOYMENT STATUS_Females 16 years and over_In labor force,Estimate_EMPLOYMENT STATUS_Females 16 years and over_In labor force_Civilian labor force,Percent Estimate_EMPLOYMENT STATUS_Females 16 years and over_In labor force_Civilian labor force,Estimate_EMPLOYMENT STATUS_Females 16 years and over_In labor force_Civilian labor force_Employed,Percent Estimate_EMPLOYMENT STATUS_Females 16 years and over_In labor force_Civilian labor force_Employed,Estimate_EMPLOYMENT STATUS_Own children of the householder under 6 years,Percent Estimate_EMPLOYMENT STATUS_Own children of the householder under 6 years,Estimate_EMPLOYMENT STATUS_Own children of the householder under 6 years_All parents in family in labor force,Percent Estimate_EMPLOYMENT STATUS_Own children of the householder under 6 years_All parents in family in labor force,Estimate_EMPLOYMENT STATUS_Own children of the householder 6 to 17 years,Percent Estimate_EMPLOYMENT STATUS_Own children of the householder 6 to 17 years,Estimate_EMPLOYMENT STATUS_Own children of the householder 6 to 17 years_All parents in family in labor force,Percent Estimate_EMPLOYMENT STATUS_Own children of the householder 6 to 17 years_All parents in family in labor force,Estimate_COMMUTING TO WORK_Workers 16 years and over,Percent Estimate_COMMUTING TO WORK_Workers 16 years and over,"Estimate_COMMUTING TO WORK_Workers 16 years and over_Car, truck, or van -- drove alone","Percent Estimate_COMMUTING TO WORK_Workers 16 years and over_Car, truck, or van -- drove alone","Estimate_COMMUTING TO WORK_Workers 16 years and over_Car, truck, or van -- carpooled","Percent Estimate_COMMUTING TO WORK_Workers 16 years and over_Car, truck, or van -- carpooled",Estimate_COMMUTING TO WORK_Workers 16 years and over_Public transportation (excluding taxicab),Percent Estimate_COMMUTING TO WORK_Workers 16 years and over_Public transportation (excluding taxicab),Estimate_COMMUTING TO WORK_Workers 16 years and over_Walked,Percent Estimate_COMMUTING TO WORK_Workers 16 years and over_Walked,Estimate_COMMUTING TO WORK_Workers 16 years and over_Other means,Percent Estimate_COMMUTING TO WORK_Workers 16 years and over_Other means,Estimate_COMMUTING TO WORK_Workers 16 years and over_Worked at home,Percent Estimate_COMMUTING TO WORK_Workers 16 years and over_Worked at home,Estimate_COMMUTING 